#Welcome to AI Based Photo Organiser 
# Microsoft Hackathon 2021
#Project by Tushar Dhadiwal (ASCII)

This Project will easily classify your photos into separate folders based on what is in the photo!

We have used a model trained to identify over 80 classes :


person
bicycle
car
motorbike
aeroplane
bus
train
truck
boat
traffic light
fire hydrant
stop sign
parking meter
bench
bird
cat
dog
horse
sheep
cow
elephant
bear
zebra
giraffe
backpack
umbrella
handbag
tie
suitcase
frisbee
skis
snowboard
sports ball
kite
baseball bat
baseball glove
skateboard
surfboard
tennis racket
bottle
wine glass
cup
fork
knife
spoon
bowl
banana
apple
sandwich
orange
broccoli
carrot
hot dog
pizza
donut
cake
chair
sofa
pottedplant
bed
diningtable
toilet
tvmonitor
laptop
mouse
remote
keyboard
cell phone
microwave
oven
toaster
sink
refrigerator
book
clock
vase
scissors
teddy bear
hair drier
toothbrush


# Clone Darknet Repository

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

# Setup Darknet config based on GPU

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
%env compute_capability=60
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i "s/ARCH= -gencode arch=compute_60,code=sm_60/ARCH= -gencode arch=compute_${compute_capability},code=sm_${compute_capability}/g" Makefile

#Verify CUDA Availability

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version
!nvidia-smi

#Lets Build Darknet!!

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
%cd /content/darknet/
!make

# Download weights for model to Identify Images
YOLOv4 has been trained already on the coco dataset which has 80 classes that it can predict. We will grab these pretrained weights so that we can run YOLOv4 on these pretrained classes and get detections.

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

# Run Detections to Classify images
Darknet is now built and ready to run detections using YOLOv4 in the cloud! 
Now We will Mount Google Drive for inpute images



In [14]:
#In this section we will run darknet on images from google drive folder and store results in another folder
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [100]:
%cd /content/drive/MyDrive/Hack2021/input/ 
!ls -d $PWD/* > /content/drive/MyDrive/Hack2021/ip.txt
!cat /content/drive/MyDrive/Hack2021/ip.txt

/content/drive/MyDrive/Hack2021/input
ls: cannot access '/content/drive/MyDrive/Hack2021/input/*': No such file or directory


In [98]:
!pwd
%cd /content/drive/MyDrive/Hack2021/darknet
!chmod +x darknet
!./darknet detector test -save_labels -dont_show cfg/coco.data cfg/yolov4.cfg yolov4.weights < /content/drive/MyDrive/Hack2021/ip.txt

/content/drive/MyDrive/Hack2021/input
/content/drive/MyDrive/Hack2021/darknet
 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    608 x 608 x   3 ->  608 x 608 x  32 0.639 BF
   1 conv     64       3 x 3/ 2    608 x 608 x  32 ->  304 x 304 x  64 3.407 BF
   2 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   3 route  1 		                           ->  304 x 304 x  64 
   4 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   5 conv     32       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  32 0.379 BF
   6 conv     64       3 x 3/ 1    304 x 304 x  32 ->  304 x 304 x  64 3.407 BF
   7 Sho

In [99]:
#Now we will read label files and move images to respective class folders
%cd /content/drive/MyDrive/Hack2021/input/
import os
import re
from pathlib import Path
import collections

with open('/content/drive/MyDrive/Hack2021/darknet/data/coco.names') as f:
    class_list = f.read().splitlines()

directory='/content/drive/MyDrive/Hack2021/input/'
Justdetfiles = [f for f in os.listdir(directory) if re.match(r'[0-9]+.*\.txt', f)]
detfiles = [os.path.abspath(os.path.join(directory, p)) for p in os.listdir(directory) if p.endswith(('txt'))]


for idx,detfile in enumerate(detfiles):
  with open(detfile) as dets:
    result=dets.read().splitlines()
    target_foldername=class_list[int(result[0].split(" ")[0])]
    Path("/content/drive/MyDrive/Hack2021/output/"+target_foldername).mkdir(parents=True, exist_ok=True)
    newfilename=Justdetfiles[idx].split(".")[0]+".jpeg"
    srcFile=str(detfile.split(".")[0])+".jpeg"
    destFile="/content/drive/MyDrive/Hack2021/output/"+target_foldername+"/"+newfilename
    os.remove(detfile)
    os.rename(srcFile,destFile)
    print(target_foldername +" Detected in file:"+ srcFile + " Moving to :"+ destFile )


/content/drive/MyDrive/Hack2021/input
elephant Detected in file:/content/drive/MyDrive/Hack2021/input/10.jpeg Moving to :/content/drive/MyDrive/Hack2021/output/elephant/10.jpeg
elephant Detected in file:/content/drive/MyDrive/Hack2021/input/11.jpeg Moving to :/content/drive/MyDrive/Hack2021/output/elephant/11.jpeg
aeroplane Detected in file:/content/drive/MyDrive/Hack2021/input/12.jpeg Moving to :/content/drive/MyDrive/Hack2021/output/aeroplane/12.jpeg
laptop Detected in file:/content/drive/MyDrive/Hack2021/input/13.jpeg Moving to :/content/drive/MyDrive/Hack2021/output/laptop/13.jpeg
cell phone Detected in file:/content/drive/MyDrive/Hack2021/input/14.jpeg Moving to :/content/drive/MyDrive/Hack2021/output/cell phone/14.jpeg
dog Detected in file:/content/drive/MyDrive/Hack2021/input/1.jpeg Moving to :/content/drive/MyDrive/Hack2021/output/dog/1.jpeg
dog Detected in file:/content/drive/MyDrive/Hack2021/input/2.jpeg Moving to :/content/drive/MyDrive/Hack2021/output/dog/2.jpeg
person Dete

Thank you for viewing ML Based Image Organiser
Email tudhadiw@Microsoft.com for Queries/Feedback